Import and file loading

In [0]:
filename = 's3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json'

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql import Row # this will let us manipulate rows with spark sql
import pandas as pd


We create a df from the json file and we examine the schema

In [0]:
df = spark.read.format('json').load(filename)
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- developer: string (nullable = true)
 |    |-- discount: string (nullable = true)
 |    |-- genre: string (nullable = true)
 |    |-- header_image: string (nullable = true)
 |    |-- initialprice: string (nullable = true)
 |    |-- languages: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- negative: long (nullable = true)
 |    |-- owners: string (nullable = true)
 |    |-- platforms: struct (nullable = true)
 |    |    |-- linux: boolean (nullable = true)
 |    |    |-- mac: boolean (nullable = true)
 |    |    |-- windows: boolean (nullable = true)
 |    |-- positive: long (nullable = true)
 |    |-- price: string (nullable = true)
 |    |-- publisher: string (nullable = true)
 |    |-- release_date: string (nullable = true)
 |    |-

In [0]:
df.count()

Out[4]: 55691

In [0]:
df.show(5)

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
+--------------------+-------+
only showing top 5 rows



# Part 1 Analysis at the "macro" level

1) Which publisher has released the most games on Steam?

2) What are the best rated games?

3) Are there years with more releases? Were there more or fewer game releases during the Covid, for example?

4) How are the prices distributed? Are there many games with a discount?

5) What are the most represented languages?

6) Are there many games prohibited for children under 16/18?

#### 1. 1 Which publisher has released the most games on Steam?

In [0]:

df = df.withColumn('publisher', F.col('data').getField('publisher'))
df.show(5)

+--------------------+-------+--------------------+
|                data|     id|           publisher|
+--------------------+-------+--------------------+
|{10, [Multi-playe...|     10|               Valve|
|{1000000, [Single...|1000000|PsychoFlux Entert...|
|{1000010, [Single...|1000010|Team17, NEXT Studios|
|{1000030, [Multi-...|1000030| Vertigo Gaming Inc.|
|{1000040, [Single...|1000040|       DoubleC Games|
+--------------------+-------+--------------------+
only showing top 5 rows



In [0]:
df.groupBy('publisher').count().show()

+--------------------+-----+
|           publisher|count|
+--------------------+-----+
|      Mykhail Konokh|    9|
|                TAKS|    1|
|           IR Studio|    6|
|           BBB Games|    1|
|   Schmidt Workshops|    3|
| Iceberg Interactive|   46|
|Navila Software J...|    1|
|Virtual Human Int...|    3|
|Sword Garden Studios|    1|
|     Decumanus Games|    1|
|        Labrodex Inc|    2|
|       YSY Softworks|    5|
|      HandMade Games|   37|
|         Lexip Games|    3|
|          David Vogt|    1|
|   Groliver7 Studios|    1|
|        BisquitGames|    1|
|       Tainted Games|    4|
|       Paper Pirates|    4|
|                 tyx|    1|
+--------------------+-----+
only showing top 20 rows



In [0]:
df.groupBy('publisher').count().orderBy(col('count').desc()).show()


+--------------------+-----+
|           publisher|count|
+--------------------+-----+
|      Big Fish Games|  422|
|              8floor|  202|
|                SEGA|  165|
|      Strategy First|  151|
|         Square Enix|  141|
|     Choice of Games|  140|
|       Sekai Project|  132|
|            HH-Games|  132|
|                    |  132|
|             Ubisoft|  127|
|        Laush Studio|  126|
|          THQ Nordic|  125|
|Alawar Entertainment|  107|
|  Fulqrum Publishing|  104|
|     Plug In Digital|  101|
|            Ziggurat|  100|
|     Slitherine Ltd.|   99|
|   Sokpop Collective|   99|
|    Devolver Digital|   98|
|KOEI TECMO GAMES ...|   90|
+--------------------+-----+
only showing top 20 rows



In [0]:
grouped_df = df.groupBy('publisher').count()
grouped_df.agg(max('count')).collect()

Out[27]: [Row(max(count)=422)]

In [0]:
max_count = grouped_df.agg(max('count')).collect()[0][0]

# Filter the DataFrame to get the publisher(s) with the maximum count
max_publisher_df = grouped_df.filter(col('count') == max_count)

# Collect the result
max_publisher = max_publisher_df.collect()

# Extract the publisher names
max_publisher_name = [row['publisher'] for row in max_publisher]

max_publisher_name = max_publisher_name[0]

print(f'The publisher who has released the most games on Steam is {max_publisher_name}')

The publisher who has released the most games on Steam is Big Fish Games


In [0]:
print(f'The publisher who has released the most games on Steam is {max_publisher_name}')

The publisher who has released the most games on Steam is Big Fish Games


In [0]:
max_publisher_df.show()

+--------------+-----+
|     publisher|count|
+--------------+-----+
|Big Fish Games|  422|
+--------------+-----+



In [0]:
df.select('publisher').distinct().count()

Out[31]: 29966

1. 1 Big fish is the publisher who released the most games

#### 1. 2. What are the best rated games?

We create the columns name for the name of game and 'positive' and 'negative' for the number of positive and negative reviews

If there are different releases from the same game we will count them as different games:

 - we also create the column 'release_date' with getField and 'year' with substring method;


In [0]:

df = df.withColumn('name', F.col('data').getField('name'))
df = df.withColumn('positive', F.col('data').getField('positive'))
df = df.withColumn('negative', F.col('data').getField('negative'))
df.show(5)


+--------------------+-------+--------------------+--------------------+--------+--------+
|                data|     id|           publisher|                name|positive|negative|
+--------------------+-------+--------------------+--------------------+--------+--------+
|{10, [Multi-playe...|     10|               Valve|      Counter-Strike|  201215|    5199|
|{1000000, [Single...|1000000|PsychoFlux Entert...|           ASCENXION|      27|       5|
|{1000010, [Single...|1000010|Team17, NEXT Studios|         Crown Trick|    4032|     646|
|{1000030, [Multi-...|1000030| Vertigo Gaming Inc.|Cook, Serve, Deli...|    1575|     115|
|{1000040, [Single...|1000040|       DoubleC Games|            细胞战争|       0|       1|
+--------------------+-------+--------------------+--------------------+--------+--------+
only showing top 5 rows



In [0]:
df = df.withColumn('release_date', F.col('data').getField('release_date'))
df = df.withColumn('year', substring('release_date', 1, 4))

# Show the updated DataFrame
df.show(5)

+--------------------+-------+--------------------+--------------------+--------+--------+------------+----+
|                data|     id|           publisher|                name|positive|negative|release_date|year|
+--------------------+-------+--------------------+--------------------+--------+--------+------------+----+
|{10, [Multi-playe...|     10|               Valve|      Counter-Strike|  201215|    5199|   2000/11/1|2000|
|{1000000, [Single...|1000000|PsychoFlux Entert...|           ASCENXION|      27|       5|  2021/05/14|2021|
|{1000010, [Single...|1000010|Team17, NEXT Studios|         Crown Trick|    4032|     646|  2020/10/16|2020|
|{1000030, [Multi-...|1000030| Vertigo Gaming Inc.|Cook, Serve, Deli...|    1575|     115|  2020/10/14|2020|
|{1000040, [Single...|1000040|       DoubleC Games|            细胞战争|       0|       1|  2019/03/30|2019|
+--------------------+-------+--------------------+--------------------+--------+--------+------------+----+
only showing top 5 rows

In [0]:
df.groupBy('name').count().orderBy(col('count').desc()).show()

+-------------+-----+
|         name|count|
+-------------+-----+
|        Alone|    5|
|       Aurora|    4|
|         Lost|    4|
|       Bounce|    4|
|       Escape|    4|
|        Dodge|    4|
|        Arena|    3|
|         Maze|    3|
|     The Line|    3|
|      Arcadia|    3|
|         Cube|    3|
|     Spectrum|    3|
|    Fireflies|    3|
|Hide and Seek|    3|
|       Azrael|    3|
|        Surge|    3|
|       Vortex|    3|
|  Dark Matter|    3|
|    Labyrinth|    3|
|      Trapped|    3|
+-------------+-----+
only showing top 20 rows



we create a new column "rating" based on the number of positive and negative reviews

In [0]:
df = df.withColumn('rating', F. col('positive') - F.col ('negative'))
df.select('name', 'positive', 'negative', 'rating').show(5)

+--------------------+--------+--------+------+
|                name|positive|negative|rating|
+--------------------+--------+--------+------+
|      Counter-Strike|  201215|    5199|196016|
|           ASCENXION|      27|       5|    22|
|         Crown Trick|    4032|     646|  3386|
|Cook, Serve, Deli...|    1575|     115|  1460|
|            细胞战争|       0|       1|    -1|
+--------------------+--------+--------+------+
only showing top 5 rows



We obtain the name of the game corresponding to the max value in the rating column

In [0]:
df.agg(max('rating')).collect()[0][0]

Out[35]: 5156252

In [0]:
max_rating = df.agg(max('rating')).collect()[0][0]

# Filter the DataFrame to get the publisher(s) with the maximum count
best_rated_game = df.filter(col('rating') == max_rating)

# Collect the result
best_rated_game_name = best_rated_game.collect()

# Extract the publisher names
best_rated_game_name = [row['name'] for row in best_rated_game_name]


In [0]:
best_rated_game_name = best_rated_game.collect()
# Extract the publisher names
best_rated_game_name = [row['name'] for row in best_rated_game_name]

In [0]:
best_rated_game_name = best_rated_game_name[0]
print(f'The best rated game is {best_rated_game_name}')

The best rated game is Counter-Strike: Global Offensive


#### 1. 3. Year realeases: Are there years with more releases? Were there more or fewer game releases during the Covid, for example?


In [0]:
df.groupBy('year').count().orderBy(col('count').desc()).show(5)

+----+-----+
|year|count|
+----+-----+
|2021| 8823|
|2020| 8305|
|2018| 7678|
|2022| 7455|
|2019| 6968|
+----+-----+
only showing top 5 rows



In [0]:
df.groupBy('year').count().orderBy(col('count').asc()).show(5)

+----+-----+
|year|count|
+----+-----+
|2002|    1|
|1998|    1|
|2000|    2|
|1997|    2|
|1999|    3|
+----+-----+
only showing top 5 rows



In [0]:
df_year = df.groupBy('year').count().orderBy(col('count').desc())
df_year.show(10)


+----+-----+
|year|count|
+----+-----+
|2021| 8823|
|2020| 8305|
|2018| 7678|
|2022| 7455|
|2019| 6968|
|2017| 6017|
|2016| 4185|
|2015| 2576|
|2014| 1557|
|2013|  471|
+----+-----+
only showing top 10 rows



### Answer 1. 3 Covid years gathered most of the game releases.
%md
#### 2020 and 2021 were the years with the larger number of releases. It coincides with the covid years.


Distribution of year releases of the total of games in the dataset

In [0]:
df_year = df_year.toPandas()

Releases from 2000 and covid years

In [0]:
df_year.display()

year,count
2021,8823
2020,8305
2018,7678
2022,7455
2019,6968
2017,6017
2016,4185
2015,2576
2014,1557
2013,471


Databricks visualization. Run in Databricks to view.

#### 1. 4. How are the prices distributed? Are there many games with a discount?


1)We create the columns "initial price" and "price" through the getField function in order to extract the necessary 
y information from the original DataFrame.

2)We create a new column "price_difference" indicating if there was a difference between the initial price and the current price. This will tell us if there was a discount.

3)We calculate how many games show a negative difference, that means >0, in the column price_difference and we obtained our answer.



In [0]:
df = df.withColumn('initialprice', F.col('data').getField('initialprice'))
df = df.withColumn('price', F.col('data').getField('price'))
df = df.withColumn('price_difference', F. col('price') - F.col ('initialprice'))
df.select('initialprice', 'price', 'price_difference').show(5)

+------------+-----+----------------+
|initialprice|price|price_difference|
+------------+-----+----------------+
|         999|  999|             0.0|
|         999|  999|             0.0|
|        1999|  599|         -1400.0|
|        1999| 1999|             0.0|
|         199|  199|             0.0|
+------------+-----+----------------+
only showing top 5 rows



In [0]:
df.filter(col('price_difference') > 0).count()

Out[109]: 0

In [0]:
df.filter(col('price_difference') == 0).count()

Out[105]: 53173

In [0]:
number_discounted_games = df.filter(col('price_difference') < 0).count()
print(number_discounted_games)

2518


In [0]:
#Percentage of discounted games:

percentage_discounted_games = (number_discounted_games * 100) /df.count()

print(f'The percentage of discounted games over the total is {percentage_discounted_games}')


The percentage of discounted games over the total is 4.521376883158859


#### Answer: the number of discouted games is 2518. This means that 4,52% of the total of games in the dataset had a price reduction

#### 5. What are the most represented languages?

1) We create the column "languages" through the getField function in order to extract the necessary y information from the original DataFrame.

In [0]:
df = df.withColumn('languages', F.col('data').getField('languages'))

In [0]:
df.select('languages').show(10)

+--------------------+
|           languages|
+--------------------+
|English, French, ...|
|English, Korean, ...|
|Simplified Chines...|
|             English|
|  Simplified Chinese|
|Simplified Chines...|
|Japanese, Simplif...|
|English, Simplifi...|
|             English|
|English, Simplifi...|
+--------------------+
only showing top 10 rows



Since each game can be available in more than one language we split the values in each of the column 'languages' so that we can count them.

In [0]:
df = df.withColumn("language_count",F.size(F.split(F.col("languages"), ",")))

In [0]:
df = df.withColumn("split_languages", F.split(F.col('languages'),',\s*'))


In [0]:
df.select('languages','language_count','split_languages').show(10)

+--------------------+--------------+--------------------+
|           languages|language_count|     split_languages|
+--------------------+--------------+--------------------+
|English, French, ...|             8|[English, French,...|
|English, Korean, ...|             3|[English, Korean,...|
|Simplified Chines...|             9|[Simplified Chine...|
|             English|             1|           [English]|
|  Simplified Chinese|             1|[Simplified Chinese]|
|Simplified Chines...|             5|[Simplified Chine...|
|Japanese, Simplif...|             3|[Japanese, Simpli...|
|English, Simplifi...|             3|[English, Simplif...|
|             English|             1|           [English]|
|English, Simplifi...|             3|[English, Simplif...|
+--------------------+--------------+--------------------+
only showing top 10 rows



We create a new df with a new column "language" by applying F.explode function on the column 'split_languages' which contains a  a list of the languages available for each game. 

In df_languages, the new column "language" will contain a row per language available in each game. We will then count the number occurrences of each language to know which are the most represented languages. We create a new df, because this method creates new rows, this way we can keep the original dataframe. 

In [0]:
df_languages = df.withColumn('language', F.explode(F.col('split_languages')))


In [0]:
df_languages.select('languages','language_count','split_languages','language').show(20)

+--------------------+--------------+--------------------+-------------------+
|           languages|language_count|     split_languages|           language|
+--------------------+--------------+--------------------+-------------------+
|English, French, ...|             8|[English, French,...|            English|
|English, French, ...|             8|[English, French,...|             French|
|English, French, ...|             8|[English, French,...|             German|
|English, French, ...|             8|[English, French,...|            Italian|
|English, French, ...|             8|[English, French,...|    Spanish - Spain|
|English, French, ...|             8|[English, French,...| Simplified Chinese|
|English, French, ...|             8|[English, French,...|Traditional Chinese|
|English, French, ...|             8|[English, French,...|             Korean|
|English, Korean, ...|             3|[English, Korean,...|            English|
|English, Korean, ...|             3|[English, Korea

In [0]:
df.select('split_languages').count()

Out[49]: 55691

In [0]:
df.select('split_languages').distinct().count()

Out[50]: 8623

In [0]:
df_languages = df_languages.groupBy('language').count().orderBy(col('count').desc())
df_languages.show(10)

+-------------------+-----+
|           language|count|
+-------------------+-----+
|            English|55116|
|             German|14019|
|             French|13426|
|            Russian|12922|
| Simplified Chinese|12782|
|    Spanish - Spain|12233|
|           Japanese|10368|
|            Italian| 9304|
|Portuguese - Brazil| 6750|
|             Korean| 6600|
+-------------------+-----+
only showing top 10 rows



Top 5 most frequent languages

In [0]:
top_5_languages_df = df_languages.limit(5)


In [0]:
top_5_languages_df.show()

+------------------+-----+
|          language|count|
+------------------+-----+
|           English|55116|
|            German|14019|
|            French|13426|
|           Russian|12922|
|Simplified Chinese|12782|
+------------------+-----+



In [0]:
top_5_languages = top_5_languages_df.collect()

top_5_languages_list = [row['language'] for row in top_5_languages]


# Join the list elements into a single string, separated by commas
top_5_languages_str = ', '.join(top_5_languages_list)

print(f'The 5 most represented languages in the project steam game dataset are: {top_5_languages_str}')



The 5 most represented languages in the project steam game dataset are: English, German, French, Russian, Simplified Chinese


#### Answer: the 5 most represented languages in the project steam game dataset are: English, German, French, Russian, Simplified Chinese

### 6. Are there many games prohibited for children under 16/18?

We create a new column "required_age" through the function "getField" that extracts the information in the nested schema

In [0]:
df = df.withColumn('required_age', F.col('data').getField('required_age'))

In [0]:
over_18_games = df.filter(col('required_age') > 17).count()
over_16_games = df.filter(col('required_age') > 15).count()
no_age_games = df.filter(col('required_age') == 0).count()


In [0]:
#Percentage of games according to age:

percentage_over_18_games = (over_18_games * 100) /df.count()

print(f'The percentage of games prohibited for children under 18 over the total is {percentage_over_18_games:.2f}%')


percentage_over_16_games = (over_16_games * 100) /df.count()

print(f'The percentage of games prohibited for children under 16 over the total is {percentage_over_16_games:.2f}%')

percentage_no_age_games = (no_age_games * 100) /df.count()

print(f'The percentags of games with no age requirement is {percentage_no_age_games:.2f}%')

The percentage of games prohibited for children under 18 over the total is 0.41%
The percentage of games prohibited for children under 16 over the total is 0.55%
The percentags of games with no age requirement is 98.81%


We test our results by checking if there are any null values in 'required_age' column, and by applying a "value_ counts() to this column after creating a pandas version of the DataFrame

In [0]:
df2 = df.toPandas()
df2.required_age.value_counts()

/databricks/spark/python/pyspark/sql/pandas/conversion.py:122: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unable to convert the field data. If this column is not necessary, you may consider dropping it or converting to primitive type before the conversion.
Direct cause: Nested StructType not supported in conversion to Arrow
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warn(msg)


Out[60]: 0         55030
15          264
18          223
16           38
17           38
12           32
13           26
14           10
10            7
180           4
6             4
3             3
8             3
7             2
5             1
MA 15+        1
7+            1
21+           1
20            1
9             1
35            1
Name: required_age, dtype: int64

Answer: As the results show, most  of the games in the dataset are allowed to children with no age restrictions (98.81%).  
As for the prohibited games for minors :

 -The percentage of games prohibited for children under 18  is only 0,55% over the total.

 -The percentage of games prohibited for children under 16 is 0,55% over the total.

# 2. Platform analysis


Are most games available on Windows/Mac/Linux instead?

Do certain genres tend to be preferentially available on certain platforms?

In [0]:
df = df.withColumn('platforms', F.col('data').getField('platforms'))
df.select('platforms').show(10)

+--------------------+
|           platforms|
+--------------------+
|  {true, true, true}|
|{false, false, true}|
|{false, false, true}|
| {false, true, true}|
|{false, false, true}|
| {false, true, true}|
|{false, false, true}|
|{false, false, true}|
| {false, true, true}|
|{false, false, true}|
+--------------------+
only showing top 10 rows



In [0]:
df_platforms = df.withColumn('platforms', F.col('data').getField('platforms'))\
    .withColumn('Linux', F.col('platforms.Linux'))\
    .withColumn('Mac',F.col('platforms.Mac'))\
    .withColumn('Windows',F.col('platforms.Windows'))\
    .withColumn('genre',F.col('data').getField('genre'))\
    .drop('id')    

In [0]:
df_platforms.groupBy('Linux').count().show()
df_platforms.groupBy('Mac').count().show()
df_platforms.groupBy('Windows').count().show()

+-----+-----+
|Linux|count|
+-----+-----+
| true| 8458|
|false|47233|
+-----+-----+

+-----+-----+
|  Mac|count|
+-----+-----+
| true|12770|
|false|42921|
+-----+-----+

+-------+-----+
|Windows|count|
+-------+-----+
|   true|55676|
|  false|   15|
+-------+-----+



In [0]:
genres_mac = df_platforms.filter(F.col('Mac')== "True").select('genre').distinct().count()
genres_linux = df_platforms.filter(F.col('Linux')== "True").select('genre').distinct().count()
genres_windows = df_platforms.filter(F.col('Windows')== "True").select('genre').distinct().count()

print(f'Number of distinct genres for Linux: {genres_mac}')
print(f'Number of distinct genres for Mac: {genres_linux}')
print(f'Number of distinct genres for Windows: {genres_windows}')



Number of distinct genres for Linux: 844
Number of distinct genres for Mac: 595
Number of distinct genres for Windows: 1832


In [0]:
print('Total number of games: {}'.format(df.select('id').distinct().count()))
print('Availability of games according to platforms:')
print()
print('Linux: {}'.format(df_platforms.filter(F.col('Linux')== "True").count()))
print('Mac: {}'.format(df_platforms.filter(F.col('Mac')== "True").count()))
print('Windows: {}'.format(df_platforms.filter(F.col('Windows')== "True").count()))

Total number of games: 55691
Availability of games according to platforms:

Linux: 8458
Mac: 12770
Windows: 55676


In [0]:
df_mac.select('genre').show()

+--------------------+
|               genre|
+--------------------+
|              Action|
|Action, Indie, Si...|
|Action, Adventure...|
|       Casual, Indie|
|Action, Indie, RP...|
|Action, Casual, I...|
|    Adventure, Indie|
|Action, Adventure...|
|Action, Adventure...|
|Adventure, Casual...|
|     Indie, Strategy|
|Action, Massively...|
|Adventure, Indie,...|
|               Indie|
|Casual, Indie, St...|
|Casual, Simulatio...|
|Adventure, Indie,...|
|Adventure, Indie,...|
|Casual, Indie, Ra...|
|       Action, Indie|
+--------------------+
only showing top 20 rows



In [0]:
df_mac = df_mac.withColumn("split_genres", F.split(F.col('genre'),',\s*'))
df_mac_2 = df_mac.withColumn('genre_2', F.explode(F.col('split_genres')))

df_linux = df_linux.withColumn("split_genres", F.split(F.col('genre'),',\s*'))
df_linux_2 = df_linux.withColumn('genre_2', F.explode(F.col('split_genres')))

df_windows = df_windows.withColumn("split_genres", F.split(F.col('genre'),',\s*'))
df_windows_2 = df_windows.withColumn('genre_2', F.explode(F.col('split_genres')))


In [0]:
df_mac_2.select('genre_2','name').show(5)

+----------+--------------------+
|   genre_2|                name|
+----------+--------------------+
|    Action|      Counter-Strike|
|    Action|Cook, Serve, Deli...|
|     Indie|Cook, Serve, Deli...|
|Simulation|Cook, Serve, Deli...|
|  Strategy|Cook, Serve, Deli...|
+----------+--------------------+
only showing top 5 rows



In [0]:
genres_mac = df_mac_2.select('genre_2').collect()
genres_linux = df_linux_2.select('genre_2').collect()
genres_windows = df_windows_2.select('genre_2').collect()

# Extract the genre names in each row to make a list per platform
genres_mac = [row['genre_2'] for row in genres_mac]
genres_linux = [row['genre_2'] for row in genres_linux]
genres_windows = [row['genre_2'] for row in genres_windows]



In [0]:
genres_mac_count = df_mac_2.select('genre_2').distinct().count()
genres_linux_count = df_linux_2.select('genre_2').distinct().count()
genres_windows_count = df_windows_2.select('genre_2').distinct().count()

print(f'Number of distinct genres for Mac: {genres_mac_count}')
print(f'Number of distinct genres for Linux: {genres_linux_count}')
print(f'Number of distinct genres for Windows: {genres_windows_count}')

Number of distinct genres for Mac: 28
Number of distinct genres for Linux: 27
Number of distinct genres for Windows: 29


In [0]:
genres_mac

Out[74]: ['Action',
 'Action',
 'Indie',
 'Simulation',
 'Strategy',
 'Action',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Indie',
 'Action',
 'Indie',
 'RPG',
 'Simulation',
 'Early Access',
 'Action',
 'Casual',
 'Indie',
 'Massively Multiplayer',
 'Adventure',
 'Indie',
 'Action',
 'Adventure',
 'Indie',
 'Action',
 'Adventure',
 'Indie',
 'Adventure',
 'Casual',
 'Indie',
 'RPG',
 'Simulation',
 'Indie',
 'Strategy',
 'Action',
 'Massively Multiplayer',
 'Simulation',
 'Adventure',
 'Indie',
 'RPG',
 'Indie',
 'Casual',
 'Indie',
 'Strategy',
 'Casual',
 'Simulation',
 'Strategy',
 'Adventure',
 'Indie',
 'RPG',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Indie',
 'Racing',
 'Sports',
 'Action',
 'Indie',
 'Indie',
 'Sports',
 'Indie',
 'Action',
 'Indie',
 'Early Access',
 'Adventure',
 'Indie',
 'RPG',
 'Casual',
 'Indie',
 'Indie',
 'Adventure',
 'Casual',
 'Indie',
 'Casual',
 'Indie',
 'Sports',
 'Early Access',
 'Adventure',
 'Casual',
 'Indie',
 'Simulation',
 'Adventure'

In [0]:

# Count the occurrences of each genre in the Mac DataFrame
df_mac_genres = df_mac_2.groupBy('genre_2').agg(F.count('*').alias('Mac_Count'))

# Do the same for the Linux DataFrame
df_linux_genres = df_linux_2.groupBy('genre_2').agg(F.count('*').alias('Linux_Count'))

# And for the Windows DataFrame
df_windows_genres = df_windows_2.groupBy('genre_2').agg(F.count('*').alias('Windows_Count'))


In [0]:
df_mac_genres.show(5)

+--------------------+---------+
|             genre_2|Mac_Count|
+--------------------+---------+
|           Education|       56|
|Massively Multipl...|      270|
|      Sexual Content|       13|
|           Adventure|     5039|
|              Sports|      506|
+--------------------+---------+
only showing top 5 rows



In [0]:
df_mac_2.select('genre_2').count()

Out[77]: 35957

In [0]:
# Join Mac and Linux DataFrames on 'genre_2'
df_mac_linux = df_mac_genres.join(df_linux_genres, on='genre_2')#, how='outer')

# Join the result with the Windows DataFrame
df_all_platforms = df_mac_linux.join(df_windows_genres, on='genre_2')#, how='outer')

# Fill any missing values (if a genre is not present in a platform) with 0
df_all_platforms = df_all_platforms.fillna(0)

# Show the combined DataFrame
df_all_platforms.show(5)#(truncate=False)


+--------------------+---------+-----------+-------------+
|             genre_2|Mac_Count|Linux_Count|Windows_Count|
+--------------------+---------+-----------+-------------+
|           Education|       56|         19|          317|
|Massively Multipl...|      270|        164|         1459|
|      Sexual Content|       13|          7|           54|
|           Adventure|     5039|       3302|        21427|
|              Sports|      506|        287|         2665|
+--------------------+---------+-----------+-------------+
only showing top 5 rows



In [0]:
# Calculate the total count of genres for each platform
total_mac = df_all_platforms.select(F.sum('Mac_Count')).collect()[0][0]
total_linux = df_all_platforms.select(F.sum('Linux_Count')).collect()[0][0]
total_windows = df_all_platforms.select(F.sum('Windows_Count')).collect()[0][0]

# Add percentage columns for each platform
df_all_platforms = df_all_platforms.withColumn(
    'Mac_Percentage', F.round((F.col('Mac_Count') / total_mac) * 100, 2)
)
df_all_platforms = df_all_platforms.withColumn(
    'Linux_Percentage', F.round((F.col('Linux_Count') / total_linux) * 100, 2)
)

df_all_platforms = df_all_platforms.withColumn(
    'Windows_Percentage', F.round((F.col('Windows_Count') / total_windows) * 100, 2)
)

# Show the resulting DataFrame
df_all_platforms.show()

+--------------------+---------+-----------+-------------+--------------+----------------+------------------+
|             genre_2|Mac_Count|Linux_Count|Windows_Count|Mac_Percentage|Linux_Percentage|Windows_Percentage|
+--------------------+---------+-----------+-------------+--------------+----------------+------------------+
|           Education|       56|         19|          317|          0.16|            0.08|               0.2|
|Massively Multipl...|      270|        164|         1459|          0.75|            0.68|              0.93|
|      Sexual Content|       13|          7|           54|          0.04|            0.03|              0.03|
|           Adventure|     5039|       3302|        21427|         14.02|           13.74|             13.63|
|              Sports|      506|        287|         2665|          1.41|            1.19|               1.7|
|    Audio Production|       41|          7|          193|          0.11|            0.03|              0.12|
|    Video

In [0]:
display(df_all_platforms.orderBy(col('Mac_Percentage').desc()).limit(5))


genre_2,Mac_Count,Linux_Count,Windows_Count,Mac_Percentage,Linux_Percentage,Windows_Percentage
Indie,9935,6978,39676,27.63,29.04,25.24
Casual,5130,3305,22082,14.27,13.75,14.05
Adventure,5039,3302,21427,14.02,13.74,13.63
Action,4564,3379,23755,12.69,14.06,15.11
Strategy,3005,1826,10892,8.36,7.6,6.93


In [0]:
top_mac = df_all_platforms.orderBy(F.col('Mac_Percentage').desc()).limit(5).toPandas()
top_linux = df_all_platforms.orderBy(F.col('Linux_Percentage').desc()).limit(5).toPandas()
top_windows = df_all_platforms.orderBy(F.col('Windows_Percentage').desc()).limit(5).toPandas()

display(top_mac)
display(top_linux)
display(top_windows)

genre_2,Mac_Count,Linux_Count,Windows_Count,Mac_Percentage,Linux_Percentage,Windows_Percentage
Indie,9935,6978,39676,27.63,29.04,25.24
Casual,5130,3305,22082,14.27,13.75,14.05
Adventure,5039,3302,21427,14.02,13.74,13.63
Action,4564,3379,23755,12.69,14.06,15.11
Strategy,3005,1826,10892,8.36,7.6,6.93


Databricks visualization. Run in Databricks to view.

genre_2,Mac_Count,Linux_Count,Windows_Count,Mac_Percentage,Linux_Percentage,Windows_Percentage
Indie,9935,6978,39676,27.63,29.04,25.24
Action,4564,3379,23755,12.69,14.06,15.11
Casual,5130,3305,22082,14.27,13.75,14.05
Adventure,5039,3302,21427,14.02,13.74,13.63
Strategy,3005,1826,10892,8.36,7.6,6.93


genre_2,Mac_Count,Linux_Count,Windows_Count,Mac_Percentage,Linux_Percentage,Windows_Percentage
Indie,9935,6978,39676,27.63,29.04,25.24
Action,4564,3379,23755,12.69,14.06,15.11
Casual,5130,3305,22082,14.27,13.75,14.05
Adventure,5039,3302,21427,14.02,13.74,13.63
Strategy,3005,1826,10892,8.36,7.6,6.93


Visualization

In [0]:
df_genres_pandas = df_all_platforms.toPandas()

In [0]:

import plotly.express as px
fig = px.bar(x=df_genres_pandas['genre_2'], y= (df_genres_pandas['Mac_Percentage']))

# Show the figure
fig.show()

In [0]:
import plotly.express as px
fig = px.bar(x=df_genres_pandas['genre_2'], y= (df_genres_pandas['Linux_Percentage']))

# Show the figure
fig.show()

In [0]:
fig = px.bar(x=df_genres_pandas['genre_2'], y= (df_genres_pandas['Windows_Percentage']))


# Show the figure
fig.show()


In [0]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add bar for Mac
fig.add_trace(go.Bar(
    x=df_genres_pandas['genre_2'],
    y=df_genres_pandas['Mac_Percentage'],
    name='Mac',
    marker_color='#4CAF50'
))

# Add bar for Linux
fig.add_trace(go.Bar(
    x=df_genres_pandas['genre_2'],
    y=df_genres_pandas['Linux_Percentage'],
    name='Linux',
    marker_color='#FFC107'
))

# Add bar for Windows
fig.add_trace(go.Bar(
    x=df_genres_pandas['genre_2'],
    y=df_genres_pandas['Windows_Percentage'],
    name='Windows',
    marker_color='#2196F3'
))

# Update layout
fig.update_layout(
    title='Genres by Percentage per Platform',
    xaxis_title='Genre',
    yaxis_title='Percentage',
    barmode='group',  # Grouping the bars
    template='plotly_white',  
)

# Show the figure
fig.show()


In [0]:
# Create a figure
fig = go.Figure()

# Add bar for Mac
fig.add_trace(go.Bar(
    x=top_mac['genre_2'],
    y=top_mac['Mac_Percentage'],
    name='Mac',
    marker_color='#4CAF50'
))

# Add bar for Linux
fig.add_trace(go.Bar(
    x=top_linux['genre_2'],
    y=top_linux['Linux_Percentage'],
    name='Linux',
    marker_color='#FFC107'
))

# Add bar for Windows
fig.add_trace(go.Bar(
    x=top_windows['genre_2'],
    y=top_windows['Windows_Percentage'],
    name='Windows',
    marker_color='#2196F3'
))

# Update layout
fig.update_layout(
    title='Top 5 genres by percentage per platform',
    xaxis_title='Genre',
    yaxis_title='Percentage',
    barmode='group',  # Grouping the bars
    template='plotly_white',  # Optional: Change the template to 'plotly_white'
)

# Show the figure
fig.show()

#### Answer: the results obtained after creating a dataframe containing the number of occurrences of each genre per platform show that:
- No genre tends to be more present in one platform than in others

- Even if there are small percentage differences the top five of genres per platform is the same
	
- The results obtained after creating the new data frame and calculating the percentage of games of each genre per platform can be visualised in the graphs above.

- It is worth underlining that many games are tagged with more than one genre.
